# MPI SYSTEM CLASS

More context here:

https://www.codingame.com/playgrounds/349/introduction-to-mpi/mpi_comm_world-size-and-ranks

- MPI_Init(&argc, &argv): Communicator set up 

- MPI_Comm_size(MPI_COMM_WORLD, &size): Get the number of processes

- MPI_Comm_rank(MPI_COMM_WORLD, &rank): Get the rank of the process

- MPI_Finalize(): Finalize the MPI environment

![image.png](SendReceive.png)

# MPI SEND CLASS

- int **MPI_Sendrecv**(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
    int dest, int sendtag, void *recvbuf, int recvcount,
    MPI_Datatype recvtype, int source, int recvtag,
    MPI_Comm comm, MPI_Status *status): 

Sends and receives a message with the right choice of source and destination. Then you always need a “pair” to communicate with. If not, then you need to use “MPI_PROC_NULL”

- int **MPI_Send**(const void* buf, int count, MPI_Datatype datatype, int dest,int tag, MPI_Comm comm) 

- int **MPI_Ssend**(const void *buf, int count, MPI_Datatype datatype, int dest, int tag, MPI_Comm comm)

• ”S” for “Synchronous”, meaning that the receiver is always forced to send an acknowledge. It will not avoid deadlocks. In this case, all unsafe operations should always deadlock, helping you out to debug and write “safer” code

- int **MPI_Bsend**(const void *buf, int count, MPI_Datatype datatype, int dest, int tag, MPI_Comm comm)

User is responsible for allocating large enough buffers

- int MPI_Isend(const void *buf, int count, MPI_Datatype datatype, int dest,
int tag, MPI_Comm comm, MPI_Request *request)

Immediate or Incomplete. “Here is my data, please send it forward as I instruct” or “I am expecting certain type of data to come to this provided buffer space”.

# MPI RECEIVE CLASS

- int MPI_Recv(void* buf, int count, MPI_Datatype datatype, int source,int tag, MPI_Comm comm, MPI_Status *status)

- int MPI_Irecv(void *buf, int count, MPI_Datatype datatype, int source, int
tag, MPI_Comm comm, MPI_Request *request)
Immediate or Incomplete
Non-blocking routines yield an MPI_Request object. This request can then
be used to query whether the operation has completed. MPI_Irecv routine
does not yield an MPI_Status object. This is because the status object
describes the actually received data, and at the completion of
the MPI_Irecv call there is no received data yet.

MPI_Info info;
MPI_Win window;
MPI_Win_create( /* size info */, info, comm, &memory, &window );
// do put and get calls
MPI_Win_free( &window );

One sided communication typical workflow

MPI_Info info;

MPI_Win window;

MPI_Win_create( /* size info */, info, comm, &memory, &window );

// do put and get calls

MPI_Win_free( &window );